In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt

In [ ]:
# Define functions

# Read in data, requires filepath
def read_data(file_path):
    data = pd.read_csv(file_path, header=None)
    linear = data.iloc[:, 9:12].values
    velocity = data.iloc[:, 5:8].values
    return linear, velocity

# define filter characteristics
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    return y

# differentiation of velocity data
def derive_rotational_acceleration(velocity, fs):
    h = 1 / fs
    num_samples, num_axes = velocity.shape
    rot_acceleration = np.zeros((num_samples, num_axes))

    for i in range(2, num_samples - 2):
        for j in range(num_axes):
            rot_acceleration[i, j] = (-velocity[i + 2, j] + 8 * velocity[i + 1, j] - 8 * velocity[i - 1, j] + velocity[i - 2, j]) / (12 * h)

    for j in range(num_axes):
        rot_acceleration[0, j] = (velocity[1, j] - velocity[0, j]) / h
        rot_acceleration[1, j] = (velocity[2, j] - velocity[0, j]) / (2 * h)
        rot_acceleration[-1, j] = (velocity[-1, j] - velocity[-2, j]) / h
        rot_acceleration[-2, j] = (velocity[-1, j] - velocity[-3, j]) / (2 * h)

    return rot_acceleration

# Transformation to head com
def transform_to_head_com(linear, velocity, rot_acceleration, x):
    num_samples = linear.shape[0]
    trans_linear = np.zeros((num_samples, 4))

    for i in range(num_samples):
        w = velocity[i, :]
        dw = rot_acceleration[i, :]
        lin_acc = linear[i, :]
        cross_rot = np.cross(w, x)
        tranlin = lin_acc + np.cross(dw, x) + np.cross(cross_rot, w)
        trans_linear[i, :3] = tranlin / 9.81
        trans_linear[i, 3] = np.linalg.norm(trans_linear[i, :3])

    return trans_linear

# summarise kinematics
def summarize_peak_kinematics(trans_linear, rot_acceleration):
    PLA = np.max(trans_linear[:, 3])
    PRA = np.max(np.linalg.norm(rot_acceleration, axis=1))
    return PLA, PRA


In [ ]:
# Define main script

def process_kinematics():
    fs = 3200  # Sampling rate
    cf = 200   # Cut-off frequency
    x = np.array([-0.04584, -0.032, -0.06228])  # Displacement vector

    file_dir = '.'  # Assuming CSV files are in the current directory
    files = [f for f in os.listdir(file_dir) if f.endswith('.csv')]

    PLA_list = []
    PRA_list = []

    for file in files:
        file_path = os.path.join(file_dir, file)

        linear, velocity = read_data(file_path)

        linear_filtered = butter_lowpass_filter(linear, cf, fs)
        velocity_filtered = butter_lowpass_filter(velocity, cf, fs)

        rot_acceleration = derive_rotational_acceleration(velocity_filtered, fs)
        rot_acceleration_filtered = butter_lowpass_filter(rot_acceleration, cf, fs)

        trans_linear = transform_to_head_com(linear_filtered, velocity_filtered, rot_acceleration_filtered, x)

        PLA, PRA = summarize_peak_kinematics(trans_linear, rot_acceleration_filtered)

        PLA_list.append(PLA)
        PRA_list.append(PRA)

    return PLA_list, PRA_list


In [ ]:
# Run Script
PLA_list, PRA_list = process_kinematics()

print("Peak Linear Acceleration:", PLA_list)
print("Peak Rotational Acceleration:", PRA_list)


# Output script
PLA_list.to_csv(filepath + "PLA.csv")
PRA_list.to_csv(filepath + "PRA.csv")